In [1]:
import cv2
import os

def video_to_frames(video_path, output_path, interval=1.75):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    # Get the frames per second (fps) of the input video
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Calculate the frame interval based on the desired interval in seconds
    frame_interval = int(fps * interval)
    
    # Create the output directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)

    # Initialize frame counter
    frame_count = 0
    
    while True:
        ret, frame = cap.read()
        
        if not ret:
            break  # Break the loop if we reach the end of the video
        
        # Save the frame every frame_interval frames
        if frame_count % frame_interval == 0:
            frame_filename = os.path.join(output_path, f"frame_{frame_count // frame_interval:04d}.jpg")
            cv2.imwrite(frame_filename, frame)
        
        frame_count += 1
    
    # Release the video capture object
    cap.release()

# Example usage:
video_path = "C://Users//owner//image project//Project Video.mp4"  # Replace with the path to your video file
output_path = "C://Users//owner//image project//output6" # Replace with the desired output directory

video_to_frames(video_path, output_path, interval=1.75)

In [2]:
import cv2
import matplotlib.pyplot as plt
import os

def resize_image(img):
    h, w, c = img.shape

    if w > 1000:
        new_w = 1000
        ar = w / h
        new_h = int(new_w / ar)

        img = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)

    return img
def process_frames(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Loop through each file in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            # Read the image
            img_path = os.path.join(input_folder, filename)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Process the image
            resized_img = resize_image(img)

            # Save the processed image to the output folder
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, cv2.cvtColor(resized_img, cv2.COLOR_RGB2BGR))

if __name__ == "__main__":
    input_folder = "C://Users//owner//image project//output6"
    output_folder = "C://Users//owner//image project//output2"

    process_frames(input_folder, output_folder)


In [9]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# Function to process image and detect subtitles using word-level segmentation
def detect_subtitles_word_level(input_folder_path, output_folder_path1,output_folder_path2,output_folder_path3):
    # Create a new folder if it doesn't exist to save processed images
    if not os.path.exists(output_folder_path1):
        os.makedirs(output_folder_path1)

    # Iterate through all files in the input folder
    for filename in os.listdir(input_folder_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):  # Change extensions as needed
            # Construct the full path to the input image
            input_image_path1 = os.path.join(input_folder_path, filename)

            # Read the input image
            image = cv2.imread(input_image_path1)
            original_image = image.copy()
            # Read the image
            # Calculate subtitle area (assuming it's at the bottom 1/4th of the image)
            height, width, _ = original_image.shape
            subtitle_area_start = int(3.2 * height / 4)

            # Crop the bottom quarter of the image for word detection
            bottom_quarter = original_image[subtitle_area_start:, :]
            # Convert cropped image to grayscale
            gray = cv2.cvtColor(bottom_quarter, cv2.COLOR_BGR2GRAY)
            gaussian = cv2.GaussianBlur(gray, (5, 5), 0)

            # Construct the path to save the processed image
            output_image_path1 = os.path.join(output_folder_path1, f"processed_{filename}")

            # Save the processed image to the output folder
            cv2.imwrite(output_image_path1, gaussian)

            if not os.path.exists(output_folder_path2):
                os.makedirs(output_folder_path2)

            # Perform adaptive thresholding to obtain binary image
    #for filename in os.listdir(output_folder_path1):
            if filename.endswith(".jpg") or filename.endswith(".png"):
                input_image_path2 = os.path.join(output_folder_path1, filename)
                _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)

                        # Construct the path to save the processed image
                output_image_path2 = os.path.join(output_folder_path2, f"processed_{filename}")

                        # Save the processed image to the output folder
                cv2.imwrite(output_image_path2, thresh)

                        # Perform morphological operations to clean noise and separate words
                kernel = np.ones((2, 2), np.uint8)
                morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)

                        # Find contours
                contours, _ = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                        # Dilate the contours to increase space between letters
                dilated = cv2.dilate(morph, np.ones((2, 2), np.uint8), iterations=6)

                        # Find contours after dilation
                contours_dilated, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                        # Get bounding boxes around words after dilation
                word_boxes_dilated = [cv2.boundingRect(contour) for contour in contours_dilated]

                        # Draw bounding boxes around words with increased space between letters
                for x, y, w, h in word_boxes_dilated:
                            # Adjust coordinates based on the cropped region
                    cv2.rectangle(bottom_quarter, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Green bounding boxes

                        # Convert image from BGR to RGB for Matplotlib display
                bottom_quarter_rgb = cv2.cvtColor(bottom_quarter, cv2.COLOR_BGR2RGB)

                        # Extract upper portion of the image (excluding subtitle area)
                upper_part = original_image[:subtitle_area_start, :]

                        # Combine the upper and lower parts to create a full image
                full_image = np.vstack((upper_part, bottom_quarter))

                        # Assuming 'sorted_contours_lines' and 'img2' are defined
                img2 = bottom_quarter.copy()  # Make a copy of the full image
                contours, hierarchy = cv2.findContours(dilated.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

                        # Sort contours based on the y-coordinate of the bounding rectangle's top-left corner
                sorted_contours_lines = sorted(contours, key=lambda ctr: cv2.boundingRect(ctr)[1])
                        # The result is a list of contours sorted based on their vertical position
                        # Group contours (words) into clusters based on vertical proximity
                word_groups = []
                current_group = []
                sorted_contours_lines = sorted_contours_lines[:100]  # Limiting to the top 100 contours for example

                for ctr in sorted_contours_lines:
                    x, y, w, h = cv2.boundingRect(ctr)
                            # Assuming contours are sorted vertically
                    if len(current_group) == 0 or y - current_group[-1][1] < 20:  # Adjust the threshold for vertical proximity
                        current_group.append((x, y, w, h))
                    else:
                        word_groups.append(current_group)
                        current_group = [(x, y, w, h)]
                word_groups.append(current_group)  # Append the last group

                        # Draw rectangles around groups of words
                for group in word_groups:
                    if len(group) >= 5:  # Only draw rectangles around groups with 5 or more words
                        x_min = min(coord[0] for coord in group)
                        y_min = min(coord[1] for coord in group)
                        x_max = max(coord[0] + coord[2] for coord in group)
                        y_max = max(coord[1] + coord[3] for coord in group)
                        cv2.rectangle(img2, (x_min, y_min), (x_max, y_max), (100, 100, 300), 5)
                if not os.path.exists(output_folder_path3):
                         os.makedirs(output_folder_path3)

                                # Perform adaptive thresholding to obtain binary image
    #for filename in os.listdir(output_folder_path2):
                if filename.endswith(".jpg") or filename.endswith(".png"):
                    input_image_path3 = os.path.join(output_folder_path2, filename)

                                            # Combine the upper and lower parts to create a full image
                    full_image = np.vstack((upper_part, img2))


                                                # Construct the path to save the processed image
                    output_image_path3 = os.path.join(output_folder_path3, f"processed_{filename}")

                                                # Save the processed image to the output folder
                    cv2.imwrite(output_image_path3, full_image)

# Folder paths
input_images_folder = "C://Users//owner//image project//output6"
output_images_folder1 = "C://Users//owner//image project//output3"
output_images_folder2 = "C://Users//owner//image project//output4"
output_images_folder3 = "C://Users//owner//image project//output5"


# Call the function to process images in the input folder and save processed images in the output folder
detect_subtitles_word_level(input_images_folder, output_images_folder1,output_images_folder2,output_images_folder3)


In [10]:
import cv2
import os

def frames_to_video(input_folder, output_video_path, fps=1):
    # Get the list of frames in the input folder
    frames = [os.path.join(input_folder, frame) for frame in os.listdir(input_folder) if frame.endswith('.jpg')]

    # Sort the frames based on their file names (assuming the naming is sequential)
    frames.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))

    # Get the frame size from the first frame
    frame = cv2.imread(frames[0])
    height, width, _ = frame.shape
    frame_size = (width, height)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can change the codec as needed
    out = cv2.VideoWriter(output_video_path, fourcc, fps, frame_size)

    # Write frames to the video
    for frame_path in frames:
        frame = cv2.imread(frame_path)
        out.write(frame)

    # Release the VideoWriter object
    out.release()

if __name__ == "__main__":
    input_folder = "C://Users//owner//image project//output5"
    output_video_path = "C://Users//owner//image project//output_video.mp4"

    frames_to_video(input_folder, output_video_path)